# TAX_REBALANCER

## Overview
This function provides a tax-efficient rebalancing strategy for investment portfolios, specifically designed for use in Excel. It systematically reallocates assets across taxable and tax-advantaged accounts (such as Taxable, Roth, and Traditional IRAs) to help investors achieve their target asset allocation while minimizing the tax impact of trades. 

The rebalancing process prioritizes placing tax-inefficient assets (those that generate higher taxable income, like bonds or REITs) into tax-advantaged accounts, while allocating tax-efficient assets (such as index funds or ETFs) to taxable accounts. By doing so, the function aims to reduce current and future tax liabilities, maximize after-tax returns, and ensure that account-specific contribution or holding limits are respected.

Key features of this approach include:
- **Automated Asset Placement:** Assets are sorted by their tax efficiency, and the function automatically determines the optimal account for each asset based on available capacity and tax characteristics.
- **Constraint Handling:** The function respects the maximum allowable balances for each account type, ensuring that allocations do not exceed specified limits.
- **Delta-Based Rebalancing:** It calculates the difference between current and target allocations for each asset class, then incrementally adjusts holdings to close these gaps in a tax-aware manner.
- **Error Handling:** If inputs are invalid or constraints cannot be satisfied, the function returns a clear error message.

This tool is ideal for investors and advisors seeking to automate the complex process of tax-aware rebalancing, reduce manual effort, and avoid common pitfalls that can lead to unnecessary tax costs. It is intended for educational and planning purposes, and should not be considered tax advice.

## Usage
To use the `TAX_REBALANCER` function in Excel, enter it as a formula in a cell, specifying your asset classes, current and target allocations, tax scores, and account capacities:

```excel
=TAX_REBALANCER(asset_classes, current_alloc, target_alloc, tax_scores, taxable_cap, roth_cap, trad_cap)
```

## Arguments
|Argument|Type|Required|Description|Example|
|---|---|---|---|---|
|asset_classes|List[List[str]]|Yes|Asset class names as a list of lists, one per asset class.|[["US Stocks"],["Bonds"],["Cash"]]|
|current_alloc|List[List[float]]|Yes|Current dollar allocations for each asset in each account.|[[20000,5000,5000],[10000,2000,8000],[5000,1000,4000]]|
|target_alloc|List[List[float]]|Yes|Target total dollar allocation for each asset class.|[[30000],[20000],[10000]]|
|tax_scores|List[List[float]]|Yes|Tax efficiency score for each asset.|[[90],[60],[30]]|
|taxable_cap|float|Yes|Maximum dollars available in the taxable account.|30000|
|roth_cap|float|Yes|Maximum dollars available in the Roth account.|8000|
|trad_cap|float|Yes|Maximum dollars available in the Traditional account.|17000|

## Returns
|Returns|Type|Description|Example|
|---|---|---|---|
|Allocation|List[List[float]]|New allocation for each asset: [[taxable, roth, trad], ...]|[[25000,3000,2000],[5000,2000,13000],[0,3000,7000]]|
|Error|string|Error message if an exception occurs|"Error: Invalid input"|

## Limitations
- Assumes all assets are available in all accounts (no fund menu restrictions).
- Does not account for transaction costs or wash sale rules.
- Designed for educational and planning purposes; not tax advice.

## Benefits
- Automates tax-aware rebalancing for Excel users.
- Reduces manual effort and potential for tax-inefficient trades.
- Flexible for a variety of asset mixes and account types.

## Examples

### Rebalancing a Three-Asset Portfolio

**Sample Input:**
|Asset Class|Current Taxable|Current Roth|Current Trad|Target|Tax Score|
|---|---|---|---|---|---|
|US Stocks|20000|5000|5000|30000|90|
|Bonds|10000|2000|8000|20000|60|
|Cash|5000|1000|4000|10000|30|

**Sample Call:**
```excel
=TAX_REBALANCER(
  [["US Stocks"],["Bonds"],["Cash"]],
  [[20000,5000,5000],[10000,2000,8000],[5000,1000,4000]],
  [[30000],[20000],[10000]],
  [[90],[60],[30]],
  30000,8000,17000)
```

**Sample Output:**
|Asset Class|Taxable|Roth|Trad|
|---|---|---|---|
|US Stocks|25000|3000|2000|
|Bonds|5000|2000|13000|
|Cash|0|3000|7000|

### Allocating All Assets to Taxable Accounts

**Sample Input:**
|Asset Class|Current Taxable|Current Roth|Current Trad|Target|Tax Score|
|---|---|---|---|---|---|
|ETF1|8000|0|0|9000|10|
|ETF2|2000|0|0|3000|10|

**Sample Call:**
```excel
=TAX_REBALANCER(
  [["ETF1"],["ETF2"]],
  [[8000,0,0],[2000,0,0]],
  [[9000],[3000]],
  [[10],[10]],
  12000,1000,1000)
```

**Sample Output:**
|Asset Class|Taxable|Roth|Trad|
|---|---|---|---|
|ETF1|9000|0|0|
|ETF2|3000|0|0|

In [ ]:
# Function Implementation

def tax_rebalancer(asset_classes, current_alloc, target_alloc, tax_scores, taxable_cap, roth_cap, trad_cap):
    """
    Tax-efficient rebalancing of assets across Taxable, Roth, and Traditional accounts.
    Args:
        asset_classes (List[List[str]]): [[asset_class1], [asset_class2], ...]
        current_alloc (List[List[float]]): [[taxable, roth, trad], ...]
        target_alloc (List[List[float]]): [[target1], [target2], ...]
        tax_scores (List[List[float]]): [[score1], [score2], ...]
        taxable_cap (float): Max dollars in taxable account.
        roth_cap (float): Max dollars in Roth account.
        trad_cap (float): Max dollars in Traditional account.
    Returns:
        List[List[float]]: New allocation [[taxable, roth, trad], ...] or a string error message if an exception occurs.
    """
    try:
        def to_2d_list(x):
            if x is None:
                return []
            if isinstance(x, (str, bytes)):
                return [[x]]
            if isinstance(x, (int, float, bool)):
                return [[x]]
            if isinstance(x, list):
                if len(x) == 0:
                    return []
                if isinstance(x[0], list):
                    return x
                else:
                    return [[v] for v in x]
            return [[x]]
        def safe_float(x):
            try:
                if isinstance(x, (list, tuple)):
                    return float(x[0])
                return float(x)
            except:
                return 0.0
        # Parse inputs as 2D lists
        asset_classes = [str(row[0]).strip() for row in to_2d_list(asset_classes)]
        current_alloc = [list(map(safe_float, row)) for row in to_2d_list(current_alloc)]
        target_alloc = [safe_float(row[0]) for row in to_2d_list(target_alloc)]
        tax_scores = [safe_float(row[0]) for row in to_2d_list(tax_scores)]
        taxable_cap = safe_float(taxable_cap)
        roth_cap = safe_float(roth_cap)
        trad_cap = safe_float(trad_cap)
        n = len(asset_classes)
        # Calculate deltas
        deltas = [target_alloc[i] - sum(current_alloc[i]) for i in range(n)]
        # Sort assets by tax inefficiency (descending)
        sorted_idx = sorted(range(n), key=lambda i: -tax_scores[i])
        # Start with current allocation
        new_alloc = [row[:] for row in current_alloc]
        # Account capacities
        taxable_left = taxable_cap
        roth_left = roth_cap
        trad_left = trad_cap
        for i in range(n):
            taxable_left -= current_alloc[i][0]
            roth_left -= current_alloc[i][1]
            trad_left -= current_alloc[i][2]
        # Allocate deltas
        for idx in sorted_idx:
            delta = deltas[idx]
            if abs(delta) < 1e-6:
                continue
            # Tax-inefficient assets: fill tax-advantaged first
            if delta > 0:
                # Add to Roth, then Trad, then Taxable
                add_roth = min(delta, roth_left)
                new_alloc[idx][1] += add_roth
                roth_left -= add_roth
                delta -= add_roth
                add_trad = min(delta, trad_left)
                new_alloc[idx][2] += add_trad
                trad_left -= add_trad
                delta -= add_trad
                add_taxable = min(delta, taxable_left)
                new_alloc[idx][0] += add_taxable
                taxable_left -= add_taxable
            else:
                # Remove from Taxable, then Trad, then Roth
                remove_taxable = min(-delta, new_alloc[idx][0])
                new_alloc[idx][0] -= remove_taxable
                delta += remove_taxable
                remove_trad = min(-delta, new_alloc[idx][2])
                new_alloc[idx][2] -= remove_trad
                delta += remove_trad
                remove_roth = min(-delta, new_alloc[idx][1])
                new_alloc[idx][1] -= remove_roth
                delta += remove_roth
        # Ensure no negative allocations
        for i in range(n):
            for j in range(3):
                if new_alloc[i][j] < 0:
                    new_alloc[i][j] = 0.0
        return [row[:] for row in new_alloc]
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_demo_basic():
    asset_classes = [["US Stocks"], ["Bonds"], ["Cash"]]
    current_alloc = [[20000, 5000, 5000], [10000, 2000, 8000], [5000, 1000, 4000]]
    target_alloc = [[30000], [20000], [10000]]
    tax_scores = [[90], [60], [30]]
    taxable_cap = 30000
    roth_cap = 8000
    trad_cap = 17000
    result = tax_rebalancer(asset_classes, current_alloc, target_alloc, tax_scores, taxable_cap, roth_cap, trad_cap)
    assert isinstance(result, list)
    assert all(isinstance(row, list) and len(row) == 3 for row in result)
    assert all(all(isinstance(x, (int, float)) for x in row) for row in result)
    assert len(result) == 3
    assert abs(sum(sum(row) for row in result) - 60000) < 1e-3

def test_demo_all_taxable():
    asset_classes = [["ETF1"], ["ETF2"]]
    current_alloc = [[8000, 0, 0], [2000, 0, 0]]
    target_alloc = [[9000], [3000]]
    tax_scores = [[10], [10]]
    taxable_cap = 12000
    roth_cap = 1000
    trad_cap = 1000
    result = tax_rebalancer(asset_classes, current_alloc, target_alloc, tax_scores, taxable_cap, roth_cap, trad_cap)
    assert isinstance(result, list)
    assert all(isinstance(row, list) and len(row) == 3 for row in result)
    assert all(all(isinstance(x, (int, float)) for x in row) for row in result)
    assert len(result) == 2
    assert abs(sum(sum(row) for row in result) - 12000) < 1e-3

def test_edge_zero_delta():
    asset_classes = [["A"], ["B"]]
    current_alloc = [[1000, 2000, 3000], [4000, 5000, 6000]]
    target_alloc = [[6000], [15000]]
    tax_scores = [[80], [20]]
    taxable_cap = 10000
    roth_cap = 7000
    trad_cap = 9000
    result = tax_rebalancer(asset_classes, current_alloc, target_alloc, tax_scores, taxable_cap, roth_cap, trad_cap)
    assert isinstance(result, list)
    assert all(isinstance(row, list) and len(row) == 3 for row in result)
    assert all(all(isinstance(x, (int, float)) for x in row) for row in result)
    assert len(result) == 2
    assert abs(sum(sum(row) for row in result) - 21000) < 1e-3

def test_edge_negative_delta():
    asset_classes = [["A"], ["B"]]
    current_alloc = [[5000, 2000, 3000], [4000, 5000, 6000]]
    target_alloc = [[7000], [10000]]
    tax_scores = [[80], [20]]
    taxable_cap = 10000
    roth_cap = 7000
    trad_cap = 9000
    result = tax_rebalancer(asset_classes, current_alloc, target_alloc, tax_scores, taxable_cap, roth_cap, trad_cap)
    assert isinstance(result, list)
    assert all(isinstance(row, list) and len(row) == 3 for row in result)
    assert all(all(isinstance(x, (int, float)) for x in row) for row in result)
    assert len(result) == 2
    assert abs(sum(sum(row) for row in result) - 17000) < 1e-3

ipytest.run()

In [ ]:
# Interactive Demo
import gradio as gr

examples = [
    [
        [["US Stocks"], ["Bonds"], ["Cash"]],
        [[20000, 5000, 5000], [10000, 2000, 8000], [5000, 1000, 4000]],
        [[30000], [20000], [10000]],
        [[90], [60], [30]],
        30000, 8000, 17000
    ],
    [
        [["ETF1"], ["ETF2"]],
        [[8000, 0, 0], [2000, 0, 0]],
        [[9000], [3000]],
        [[10], [10]],
        12000, 1000, 1000
    ]
]

default_asset_classes = [["US Stocks"], ["Bonds"], ["Cash"]]
default_current_alloc = [[20000, 5000, 5000], [10000, 2000, 8000], [5000, 1000, 4000]]
default_target_alloc = [[30000], [20000], [10000]]
default_tax_scores = [[90], [60], [30]]
default_taxable_cap = 30000
default_roth_cap = 8000
default_trad_cap = 17000

demo = gr.Interface(
    fn=tax_rebalancer,
    inputs=[
        gr.Dataframe(headers=["Asset Classes"], label="Asset Classes", row_count=3, col_count=1, type="array", value=default_asset_classes),
        gr.Dataframe(headers=["Taxable", "Roth", "Trad"], label="Current Allocation", row_count=3, col_count=3, type="array", value=default_current_alloc),
        gr.Dataframe(headers=["Target"], label="Target Allocation", row_count=3, col_count=1, type="array", value=default_target_alloc),
        gr.Dataframe(headers=["Tax Score"], label="Tax Scores", row_count=3, col_count=1, type="array", value=default_tax_scores),
        gr.Number(label="Taxable Cap", value=default_taxable_cap),
        gr.Number(label="Roth Cap", value=default_roth_cap),
        gr.Number(label="Trad Cap", value=default_trad_cap),
    ],
    outputs=gr.Dataframe(headers=["Taxable", "Roth", "Trad"], label="New Allocation"),
    examples=examples,
    description="Calculate a tax-efficient rebalancing of assets across Taxable, Roth, and Traditional accounts.",
    flagging_mode="never",
)
demo.launch()